In [75]:
# CÓDIGOS OPERACIONES
BAJAR=0
SUBIR=1
COGER_PERSONA=2
DEJAR_PERSONA=3

In [76]:
# Codificación acciones:
# (BAJAR, i, j) -> El ascensor i baja j plantas
# (SUBIR, i, j) -> El ascensor i sube j plantas
# (COGER_PERSONA, i, j) -> El ascensor i coge a la persona j
# (DEJAR_PERSONA, i, j) -> El ascensor i deja a la persona j
def decodificador_acciones(acciones):
    for accion in acciones:
        if accion[0]==BAJAR or accion[0]==SUBIR:
            if accion[0]==BAJAR:
                sube_baja = " baja "
            else:
                sube_baja = " sube "
            if accion[2]>1:
                plantas = " plantas"
            else:
                plantas = " planta"
            print("El ascensor "+str(accion[1])+sube_baja+str(accion[2])+plantas)
        else:
            if accion[0]==COGER_PERSONA:
                coge_deja = " coge a "
            else:
                coge_deja = " deja a "
            lista_gente = [str(num) for num in accion[2]]
            if len(lista_gente)>1:
                gente = "las personas "
            else:
                gente = "la persona "
            print("El ascensor "+str(accion[1])+coge_deja+gente+", ".join(lista_gente))

In [77]:
#(No es muy eficiente) Quita si hay una cadena dejar-coger o coger-dejar para un mismo ascensor y persona sin que el ascensor
# suba o baje en el medio (la persona no va a irse y volver entre ambas acciones en una solución óptima)
def quita_dejar_coger_inutiles_acciones(acciones):
    index_ultimo_dejar_por_ascensor = [[-1 for _ in range(NUM_PERSONAS)] for _ in range(NUM_ASCENSORES)]
    indexes_a_borrar = set()
    for i, accion in enumerate(acciones):
        if accion[0]==DEJAR_PERSONA or accion[0]==COGER_PERSONA:
            if index_ultimo_dejar_por_ascensor[accion[1]][accion[2]]==-1:
                index_ultimo_dejar_por_ascensor[accion[1]][accion[2]]=i
            else:
                supuestamente_inverso = acciones[index_ultimo_dejar_por_ascensor[accion[1]][accion[2]]]
                if supuestamente_inverso[0] != accion[0]:
                    indexes_a_borrar.add(index_ultimo_dejar_por_ascensor[accion[1]][accion[2]])
                    indexes_a_borrar.add(i)
                    index_ultimo_dejar_por_ascensor[accion[1]][accion[2]]=-1
                else: #IMPOSIBLE, cogeríamos (respect. dejaríamos) a la misma persona en el mismo ascensor dos veces
                    index_ultimo_dejar_por_ascensor[accion[1]][accion[2]]=i
                    
        else:
            index_ultimo_dejar_por_ascensor[accion[1]] = [-1 for _ in range(NUM_PERSONAS)] #Ya no cancelables
    
    nuevas_acciones = []
    for i in range(len(acciones)):
        if not(i in indexes_a_borrar): #asi mejor para que no se corran los indices y porque del/remove/pop tienen coste lineal
            nuevas_acciones.append(acciones[i])
    
    return nuevas_acciones

In [78]:
def fusionador_acciones(acciones):
    nuevas_acciones = []
    for accion in acciones:
        if accion[0]==COGER_PERSONA or accion[0]==DEJAR_PERSONA:
            accion = (accion[0],accion[1],[accion[2]])
            
        if bool(nuevas_acciones) and (accion[0]==nuevas_acciones[-1][0] and accion[1]==nuevas_acciones[-1][1]):
            #Si es SUBIR o BAJAR suma las plantas, si es COGER o DEJAR concatena las listas de personas
            nuevas_acciones[-1] = (nuevas_acciones[-1][0],nuevas_acciones[-1][1],nuevas_acciones[-1][2]+accion[2])
        else:
            nuevas_acciones.append(accion)
    return nuevas_acciones

In [79]:
def cocina_acciones(acciones): #En este orden
    acciones1 = quita_dejar_coger_inutiles_acciones(acciones)
    acciones2 = fusionador_acciones(acciones1)
    return acciones2

In [80]:
def coste_acciones(acciones):
    coste = 0
    for action in acciones:
        if action[0]==BAJAR or action[0]==SUBIR:
            coste += action[2]
    return coste